# Introduction
Classifying survival on the Titanic is a rite of passage task on Kaggle. This titanic task may be made more tractable by gradient boosting models, which are popular choices due to their excellent performance in classification tasks. Before jumping into the code, let's dig a little into the background math.

# Mathematical background

Gradient boosting methods make predictions using decision trees under the hood. Let's take a brief look at the intuition behind these, along with some basic mathematical descriptions. Suppose we have some outcome we want to predict, $y$, which is binary, such that $y\in\{0, 1\}$. In the context of this exercise, $y$ is survival. A value of 0 indicates a passenger died while a value of 1 indicates they survived.

Now let's suppose we have several features we can use to predict $y$, such as $X_1, X_2, ...,X_p$. These could correspond to variables like age and sex. Given all possible values for each $X$, the goal of a decision tree would be to segment this feature space into $J$ distinct and non-overlapping regions, $R_1, R_2,....R_j$. Then, for each observation that falls into the region $R_j$, the tree makes the same prediction, which is the most commonly occuring class in that region.

To make this concrete, let's say that we only have one feature, $X_1$, which corresponds to passenger sex. $X_1$ consists of two values, male and female, which have been encoded as binary integers, $X_1\in\{1, 2\}$. If a decision tree attempted to predict $y$ based on $X_1$ alone, the resulting tree would be split into two regions, $R_1$ and $R_2$, corresponding to $X_1 = 1$ and $X_1 = 2$, respectively. Let's suppose that in $R_1$, which corresponds to male passengers, 75% of the individuals have died (i.e., have $y = 0$). Since the most commonly occuring value of $y$ in $R_1$ is 1, predict that all men (i.e., all observations where $X_1 = 1$) have died. If we had some new observation, $x$, with $X_1 = 1$, the tree would also predict that the corresponding $y = 0$. Here we can see the advantage and disadvantage of a simple decision tree: the results are eminently interpretable, but at the cost of accuracy.

Enter gradient boosting. Boosting involves combining $N$ decision trees, $f^1, f^2,...,f^N$, which are themselves weak learners, in order to make a better model. An initial decision tree, $f^1$, will be fit to the data and attempt to predict $y$. The second decision tree, $f^2$, will not be fit to $y$, but instead to the residuals, $r$, of $f^1$. This continues until the final tree, $f^N$. This process allows subsequent trees to improve $f$ in areas it performs poorly. Hence, boosting methods are sequential and additive. In order to slow the process (which hopefully improves learning), we can shrink each $f$ by some parameter $\lambda$. The parameters $N$, the number of trees, and $\lambda$, the shrinkage or learning rate, are inter-related and can be tuned, as we'll see later.

We can formalize the boosting method we've outlined into an algorithm:
1) Set $f(x) = 0$ and $r_i = y_i$ for all observations $i$ in the training set.

2) For $n = 1, 2, ..., N$, do:

&emsp;a) Fit tree $f^n$ to the training data $(X, r)$.

&emsp;b) Update $f$ with the shrunken output of the tree,
$$f(x) \leftarrow f(x) + \lambda f^n(x)$$

&emsp;c) Update the residuals,
$$r_i \leftarrow r_i - \lambda f^n(x_i)$$

3) Output the final boosted model,
$$f(x) = \sum_{n=1}^{N}\lambda f^n(x)$$

Keep this equation in mind, we'll come back to it later!

# Choosing a model
The number of gradient boosting algorithms available in Python is numerous. Some popular ones include built-in models in scikit-learn (AdaBoost, GradientBoost), XGBoost, LightGBM, and CatBoost.

For the purposes of this exercise we'll use Microsoft's LightGBM, a comparatively fast implementation of gradient boosting that performs quite well in classification competitions. It deals with missing values out-of-the-box, saving us from having to impute missing data (principled methods for dealing with missing data are not as mature in Python compared to R). Instead, LightGBM will ignore missing values during a split, then allocate them to the side that leads to a greater reduction in loss. Is this biased? Perhaps, but probably no less so than simple imputation methods such as taking the mean, which artificially reduces model variance.

LightGBM also handles single-column categorical features without one-hot encoding, a valuable piece of functionality that is absent from XGBoost. One-hot encoding a column with multiple categorical values can lead to unbalanced trees. By contrast, LightGBM partitions categorical variables into 2 subsets, with a maximum number of $2^{k-1}-1$ subsets for each category $k$ of a feature. This can be computationally onerous for numerous categories - the algorithm used by LightGBM is $O(n \log n)$ - but our dataset isn't that large, so we don't need to worry about it for now.

# Loading required packages and data

In [ ]:
import pandas as pd

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# increase the default figure size
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8   

# load the data
df = pd.read_csv('../input/titanic/train.csv')
df.head()

# Feature engineering
Let's with some simple feature engineering. In order to get LightGBM play nicely with categorical variables, we need to first convert them to integers.

In [ ]:
# start by encoding sex and embarked
df["Sex"].replace({'female': 0, 'male': 1}, inplace=True)
df["Embarked"] = df["Embarked"].replace({'S': 0, 'C': 1, 'Q': 2}).astype("category") 
df.head()

In [ ]:
# cabin looks sparse, so let's binarize it into cabin/no cabin.
df["Cabin"] = df["Cabin"].notnull().astype("int")
df.head()

In [ ]:
# number of siblings and parents don't seem as relevant as the family unit
df["Family"] = df["SibSp"] + df["Parch"]
df.head()

In [ ]:
# name themselves aren't informative, but salutations may be
df["Title"] = [x[0].split(" ")[-1] for x in df["Name"].str.split(".")]
df["Title"].value_counts()

In [ ]:
# those are a lot of values - let's keep the first 4 as-is and group the rest
df["Title"] = df["Title"].replace({
    'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Dr': 4, 'Rev': 4,
    'Col': 4, 'Mlle': 4, 'Major': 4, 'Jonkheer': 4, 'Don': 4,
    'Countess': 4, 'Ms': 4, 'Capt': 4, 'Mme': 4, 'Sir': 4, 'Lady': 4
}).astype("category")

df["Title"].value_counts()

In [ ]:
# we have a lot of features now - let's keep the main ones and drop the rest
keep = ["Survived", "Pclass", "Title", "Sex", "Age", "Family", "Fare", "Cabin", "Embarked"]
df = df[keep]

df.head()

# Splitting the data
We'll now split the data into train and test sets. In the train set, we'll use k-fold cross-validation to tune our model hyperparameters. We'll then take the trained model and ask it to predict survival in the test set. Given that the model has never seen the test set, this is a direct assessment of the ability of the model to generalize to new data. This strategy is referred to as nested cross-validation.

We'll use a stratified 70%-30% train-test split. Stratification will allow us to preserve the relative proportion of classes in both sets.

In [ ]:
from sklearn.model_selection import train_test_split

# split data into X and y
X = df[[x for x in df.columns if x != "Survived"]]
y = df["Survived"]

# set a random state for reproducibility
random_state = 0

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_state, stratify=y,
)

# Hyperparameter tuning with Bayesian optimization
Now that we have a set of features to work with, let's move to hyperparameter tuning in the training set. Instead a brute-force grid search, let's instead tune our parameters using Bayesian optimization. Although a grid search will find the correct answer given enough time, Bayesian optimization will allow us to approximate a reasonable answer in a fraction of the time. We'll implement this using`BayesSearchCV()` function in the scikit-optimize package.

In [ ]:
import lightgbm
from skopt import BayesSearchCV
from skopt.space import Integer, Real

# define the LGBM estimator and its search space    
estimator = lightgbm.LGBMClassifier(
    objective="binary",
    random_state=random_state,                           
)

search_spaces = {
    "learning_rate": Real(0.01, 0.1, "log-uniform"),
    "n_estimators": Integer(100, 1000),
}

Now let's run `BayesSearchCV` with 10-fold cross-validation. These folds are stratified by default to preserve class proportions. Note that with the default number of iterations (50), this function may take some time running on the Kaggle CPU.

Note that, at the time of writing, `BayesSearchCV` doesn't play nicely with the built-in version of scikit-learn (0.24.2), so if you want to run this properly, downgrade it to 0.23.2!

In [ ]:
opt = BayesSearchCV(
    estimator=estimator, search_spaces=search_spaces,
    cv=10, random_state=random_state,
)
opt.fit(X_train, y_train)

In [ ]:
train_score = opt.best_score_ * 100
test_score = opt.score(X_test, y_test) * 100

print("Best estimator:\n%s" % opt.best_estimator_)
print("Train score: %.1f%%" % train_score)
print("Test score: %.1f%%" % test_score)

# Model diagnostics
In a few minutes, Bayesian optimization has given us a reasonable set of parameters that helped the model score >80% on the train and test sets. This accuracy is well above what would be expected by chance (i.e., 50%), and the similar accuracies in the train and test splits indicates that overfitting is minimal.

Although Bayesian optimization has provided us with some wonderfully specific estimates (with a whopping 18 digits for `learning_rate`!), let's round them off to some nicer numbers and set `learning_rate=0.035` and `n_estimators=150`. These values can be nicely subsituted into our equation for generic gradient boosted models as $\lambda$ and $N$, respectively:

$$f(x) = \sum_{n=1}^{150} 0.035 f^n(x)$$

Let's take a deep dive into our model by examining some plots. First, the precision-recall curve.

In [ ]:
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve

model = lightgbm.LGBMClassifier(
    objective="binary", random_state=random_state, learning_rate=0.035, n_estimators=150
)

model.fit(X_train, y_train)

plot_precision_recall_curve(model, X_test, y_test)

The average precision of the model is 85%. Not bad! Now let's look at the confusion matrix.

In [ ]:
plot_confusion_matrix(model, X_test, y_test)

That's a pretty good confusion matrix, although there's still some room for improvement. It looks like our model is a bit too eager to predict our passengers as dead, incorrectly labelling 28 individuals who survived. If we wanted to further tune our model we could look at trying to better predict these cases.

Next let's look at feature importances.

In [ ]:
lightgbm.plot_importance(model)

It looks like fare and age are far and away the best predictors. Cabin and title don't seem to be as useful - we could perhaps remove them and re-run the model to see if we obtain better results. We could also investigate collinearity of other variables with fare, such as passenger class and port embarked.

Last but not least, let's visualize the actual model as a tree.

In [ ]:
lightgbm.create_tree_digraph(model)

This is an interesting graph as it tells you exactly how the model is making predictions. As we stated in the introduction, for any new observation $x$, LightGBM will move it through the tree based on the values of its features, $X_p$. The terminal points of the tree are leaves, and the model will predict that $x$ belongs to the most frequently occuring group at that leaf. The actual value of the leaf is the log of the probability of class membership, which is a reflection of the binary cross-entropy loss or log loss, the default loss function minimised by LightGBM for binary classification tasks.

# Conclusions
We have seen how well gradient boosting works for classification tasks such as predicting survival on the Titanic. LightGBM, in particular, worked very well with minimal feature engineering, scoring an average precision of 85% with minimal overfitting. The result was an interpretable equation and decision tree. We could clean these up a little (e.g., by changing the log probabilities in leaf nodes to 0 or 1) and then present them to anyone who wanted an interpretable model for predicting survival.

Note that the purpose of this notebook was to get a basic understanding of gradient boosting methods and their implementation. There's a lot of stuff we didn't cover, such as exploratory data visualization and applying the model to Kaggle's test set - these are exercises left to the reader!